Making a Consumer

In [0]:
#report error, throw an exception
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)


def acked(err, msg):
    """ 
        Error callback is used for generic issues for producer errors. 
        
        Parameters:
            err (err): Error flag.
            msg (str): Error message that was part of the callback.
    """
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

In [0]:
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
import json


#KAFKA variables, Move to the OS variables or configuration
# This will work in local Jupiter Notebook, but in a databrick, hiding config.py is tougher. 
confluentClusterName = "stage3talent"
confluentBootstrapServers = "pkc-ldvmy.centralus.azure.confluent.cloud:9092"
# can change topic name
confluentTopicName = "rcensus"
schemaRegistryUrl = "https://psrc-gq7pv.westus2.azure.confluent.cloud"
confluentApiKey = "YHMHG7E54LJA55XZ"
confluentSecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"
confluentRegistryApiKey = "YHMHG7E54LJA55XZ"
confluentRegistrySecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"


#Kakfa Class Setup.
c = Consumer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(uuid.uuid1()),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})

c.subscribe(['rcensus']) # subscribe to the topic


In [0]:
# Consume all messages

aString = {}
# collecting all messages to put into a data frame, panda, etc.
kafkaListDictionaries = []

# demo shows while(True):
while(True):
    try:
        msg = c.poll(timeout=10)
        print(msg)
        if msg is None:
            break
        elif msg.error():
            print("Consumer error: {}".format(msg.error()))
            break
        else:
            aString=json.loads('{}'.format(msg.value().decode('utf-8')))
            kafkaListDictionaries.append(aString)
    except Exception as e:
        print(e)
    for message in kafkaListDictionaries:
        print(message)
# if msg is None and len(kafkaListDictionaries) == 0:
#     dbutils.notebook.exit("no messages")

<cimpl.Message object at 0x7f49a50ec740>
{'State': 'Pennsylvania', 'Total_Population': '12590644', 'Public_Insurance_Alone': '2407050', 'Medicare_Alone': '657439', 'Medicaid_Alone': '1727514', 'VA_Alone': '22097', 'State_ID': '42'}
<cimpl.Message object at 0x7f49a50ec7c0>
{'State': 'Pennsylvania', 'Total_Population': '12590644', 'Public_Insurance_Alone': '2407050', 'Medicare_Alone': '657439', 'Medicaid_Alone': '1727514', 'VA_Alone': '22097', 'State_ID': '42'}
{'State': 'California', 'Total_Population': '38838726', 'Public_Insurance_Alone': '9774780', 'Medicare_Alone': '1974723', 'Medicaid_Alone': '7724092', 'VA_Alone': '75965', 'State_ID': '06'}
<cimpl.Message object at 0x7f49b58693c0>
{'State': 'Pennsylvania', 'Total_Population': '12590644', 'Public_Insurance_Alone': '2407050', 'Medicare_Alone': '657439', 'Medicaid_Alone': '1727514', 'VA_Alone': '22097', 'State_ID': '42'}
{'State': 'California', 'Total_Population': '38838726', 'Public_Insurance_Alone': '9774780', 'Medicare_Alone': '1974723', 'Medicaid_Alone': '7724092', 'VA_Alone': '75965', 'State_ID': '06'}
{'State': 'West Virginia', 'Total_Population': '1778080', 'Public_Insurance_Alone': '471938', 'Medicare_Alone': '114398', 'Medicaid_Alone': '349027', 'VA_Alone': '8513', 'State_ID': '54'}
<cimpl.Message object at 0x7f49a50ec740>
{'State': 'Pennsylvania', 'Total_Population': '12590644', 'Public_Insurance_Alone': '2407050', 'Medicare_Alone': '657439', 'Medicaid_Alone': '1727514', 'VA_Alone': '22097', 'State_ID': '42'}
{'State': 'California', 'Total_Population': '38838726', 'Public_Insurance_Alone': '9774780', 'Medicare_Alone': '1974723', 'Medicaid_Alone': '7724092', 'VA_Alone': '75965', 'State_ID': '06'}
{'State': 'West Virginia', 'Total_Population': '1778080', 'Public_Insurance_Alone': '471938', 'Medicare_Alone': '114398', 'Medicaid_Alone': '349027', 'VA_Alone': '8513', 'State_ID': '54'}
{'State': 'Utah', 'Total_Population': '3124563', 'Public_Insurance_Alone': '339098', 'Medicare_Alone': '116851', 'Medicaid_Alone': '217665', 'VA_Alone': '4582', 'State_ID': '49'}
<cimpl.Message object at 0x7f49b58693c0>
{'State': 'Pennsylvania', 'Total_Population': '12590644', 'Public_Insurance_Alone': '2407050', 'Medicare_Alone': '657439', 'Medicaid_Alone': '1727514', 'VA_Alone': '22097', 'State_ID': '42'}
{'State': 'California', 'Total_Population': '38838726', 'Public_Insurance_Alone': '9774780', 'Medicare_Alone': '1974723', 'Medicaid_Alone': '7724092', 'VA_Alone': '75965', 'State_ID': '06'}
{'State': 'West Virginia', 'Total_Population': '1778080', 'Public_Insurance_Alone': '471938', 'Medicare_Alone': '114398', 'Medicaid_Alone': '349027', 'VA_Alone': '8513', 'State_ID': '54'}
{'State': 'Utah', 'Total_Population': '3124563', 'Public_Insurance_Alone': '339098', 'Medicare_Alone': '116851', 'Medicaid_Alone': '217665', 'VA_Alone': '4582', 'State_ID': '49'}
{'State': 'New York', 'Total_Population': '19276809', 'Public_Insurance_Alone': '4542469', 'Medicare_Alone': '932578', 'Medicaid_Alone': '3585778', 'VA_Alone': '24113', 'State_ID': '36'}
<cimpl.Message object at 0x7f49a50ec740>
{'State': 'Pennsylvania', 'Total_Population': '12590644', 'Public_Insurance_Alone': '2407050', 'Medicare_Alone': '657439', 'Medicaid_Alone': '1727514', 'VA_Alone': '22097', 'State_ID': '42'}
{'State': 'California', 'Total_Population': '38838726', 'Public_Insurance_Alone': '9774780', 'Medicare_Alone': '1974723', 'Medicaid_Alone': '7724092', 'VA_Alone': '75965', 'State_ID': '06'}
{'State': 'West Virginia', 'Total_Population': '1778080', 'Public_Insurance_Alone': '471938', 'Medicare_Alone': '114398', 'Medicaid_Alone': '349027', 'VA_Alone': '8513', 'State_ID': '54'}
{'State': 'Utah', 'Total_Population': '3124563', 'Public_Insurance_Alone': '339098', 'Medicare_Alone': '116851', 'Medicaid_Alone': '217665', 'VA_Alone': '4582', 'State_ID': '49'}
{'State': 'New York', 'Total_Population': '19276809', 'Public_Insurance_Alone': '4542469', 'Medicare_Alone': '932578', 'Medicaid_Alone': '3585778', 'VA_Alone': '24113', 'State_ID': '

In [0]:
# print out results from consumer
kafkaListDictionaries

Out[66]: [{'State': 'Pennsylvania',
 'Total_Population': '12590644',
 'Public_Insurance_Alone': '2407050',
 'Medicare_Alone': '657439',
 'Medicaid_Alone': '1727514',
 'VA_Alone': '22097',
 'State_ID': '42'},
 {'State': 'California',
 'Total_Population': '38838726',
 'Public_Insurance_Alone': '9774780',
 'Medicare_Alone': '1974723',
 'Medicaid_Alone': '7724092',
 'VA_Alone': '75965',
 'State_ID': '06'},
 {'State': 'West Virginia',
 'Total_Population': '1778080',
 'Public_Insurance_Alone': '471938',
 'Medicare_Alone': '114398',
 'Medicaid_Alone': '349027',
 'VA_Alone': '8513',
 'State_ID': '54'},
 {'State': 'Utah',
 'Total_Population': '3124563',
 'Public_Insurance_Alone': '339098',
 'Medicare_Alone': '116851',
 'Medicaid_Alone': '217665',
 'VA_Alone': '4582',
 'State_ID': '49'},
 {'State': 'New York',
 'Total_Population': '19276809',
 'Public_Insurance_Alone': '4542469',
 'Medicare_Alone': '932578',
 'Medicaid_Alone': '3585778',
 'VA_Alone': '24113',
 'State_ID': '36'},
 {'State': 'District of Columbia',
 'Total_Population': '692059',
 'Public_Insurance_Alone': '154098',
 'Medicare_Alone': '17803',
 'Medicaid_Alone': '135312',
 'VA_Alone': '983',
 'State_ID': '11'},
 {'State': 'Alaska',
 'Total_Population': '711104',
 'Public_Insurance_Alone': '139179',
 'Medicare_Alone': '23270',
 'Medicaid_Alone': '111808',
 'VA_Alone': '4101',
 'State_ID': '02'},
 {'State': 'Florida',
 'Total_Population': '20897188',
 'Public_Insurance_Alone': '4367261',
 'Medicare_Alone': '1739231',
 'Medicaid_Alone': '2551095',
 'VA_Alone': '76935',
 'State_ID': '12'},
 {'State': 'South Carolina',
 'Total_Population': '4990992',
 'Public_Insurance_Alone': '1009459',
 'Medicare_Alone': '306122',
 'Medicaid_Alone': '684845',
 'VA_Alone': '18492',
 'State_ID': '45'},
 {'State': 'North Dakota',
 'Total_Population': '743105',
 'Public_Insurance_Alone': '84685',
 'Medicare_Alone': '26788',
 'Medicaid_Alone': '56303',
 'VA_Alone': '1594',
 'State_ID': '38'},
 {'State': 'Maine',
 'Total_Population': '1325025',
 'Public_Insurance_Alone': '227239',
 'Medicare_Alone': '78486',
 'Medicaid_Alone': '143635',
 'VA_Alone': '5118',
 'State_ID': '23'},
 {'State': 'Georgia',
 'Total_Population': '10321846',
 'Public_Insurance_Alone': '1851109',
 'Medicare_Alone': '542688',
 'Medicaid_Alone': '1270484',
 'VA_Alone': '37937',
 'State_ID': '13'},
 {'State': 'Alabama',
 'Total_Population': '4813429',
 'Public_Insurance_Alone': '962293',
 'Medicare_Alone': '291473',
 'Medicaid_Alone': '656093',
 'VA_Alone': '14727',
 'State_ID': '01'},
 {'State': 'New Hampshire',
 'Total_Population': '1338372',
 'Public_Insurance_Alone': '207861',
 'Medicare_Alone': '75884',
 'Medicaid_Alone': '128469',
 'VA_Alone': '3508',
 'State_ID': '33'},
 {'State': 'Oregon',
 'Total_Population': '4135531',
 'Public_Insurance_Alone': '921871',
 'Medicare_Alone': '233948',
 'Medicaid_Alone': '672968',
 'VA_Alone': '14955',
 'State_ID': '41'},
 {'State': 'Wyoming',
 'Total_Population': '571005',
 'Public_Insurance_Alone': '72850',
 'Medicare_Alone': '29086',
 'Medicaid_Alone': '41911',
 'VA_Alone': '1853',
 'State_ID': '56'},
 {'State': 'Arizona',
 'Total_Population': '7065326',
 'Public_Insurance_Alone': '1650936',
 'Medicare_Alone': '473869',
 'Medicaid_Alone': '1153664',
 'VA_Alone': '23403',
 'State_ID': '04'},
 {'State': 'Louisiana',
 'Total_Population': '4556953',
 'Public_Insurance_Alone': '1263602',
 'Medicare_Alone': '280333',
 'Medicaid_Alone': '968482',
 'VA_Alone': '14787',
 'State_ID': '22'},
 {'State': 'Indiana',
 'Total_Population': '6597701',
 'Public_Insurance_Alone': '1254559',
 'Medicare_Alone': '361041',
 'Medicaid_Alone': '876194',
 'VA_Alone': '17324',
 'State_ID': '18'},
 {'State': 'Idaho',
 'Total_Population': '1732106',
 'Public_Insurance_Alone': '298583',
 'Medicare_Alone': '89248',
 'Medicaid_Alone': '204277',
 'VA_Alone': '5058',
 'State_ID': '16'},
 {'State': 'Connecticut',
 'Total_Population': '3520172',
 'Public_Insurance_Alone': '739685',
 'Medicare_Alone': '191001',
 'Medicaid_

In [0]:
# quick ETL
# create pyspark dataframe with census topic dictionaries
# drop any NAs
# drop any duplicated messages

censusListdf = spark.createDataFrame(kafkaListDictionaries)
censusListdf = censusListdf.na.drop()
censusListdf = censusListdf.dropDuplicates()

In [0]:
# view dataframe to confirm changes
censusListdf.show()

+--------------+--------------+----------------------+--------------------+--------+----------------+--------+
Medicaid_Alone|Medicare_Alone|Public_Insurance_Alone| State|State_ID|Total_Population|VA_Alone|
+--------------+--------------+----------------------+--------------------+--------+----------------+--------+
 135312| 17803| 154098|District of Columbia| 11| 692059| 983|
 7724092| 1974723| 9774780| California| 06| 38838726| 75965|
 1727514| 657439| 2407050| Pennsylvania| 42| 12590644| 22097|
 3585778| 932578| 4542469| New York| 36| 19276809| 24113|
 349027| 114398| 471938| West Virginia| 54| 1778080| 8513|
 217665| 116851| 339098| Utah| 49| 3124563| 4582|
 111808| 23270| 139179| Alaska| 02| 711104| 4101|
 684845| 306122| 1009459| South Carolina| 45| 4990992| 18492|
 143635| 78486| 227239| Maine| 23| 1325025| 5118|
 1270484| 542688| 1851109| Georgia| 13| 10321846| 37937|
 56303| 26788| 84685| North Dakota| 38| 743105| 1594|
 2551095| 1739231| 4367261| Florida| 12| 20897188| 76935|
 1153664| 473869| 1650936| Arizona| 04| 7065326| 23403|
 128469| 75884| 207861| New Hampshire| 33| 1338372| 3508|
 968482| 280333| 1263602| Louisiana| 22| 4556953| 14787|
 672968| 233948| 921871| Oregon| 41| 4135531| 14955|
 41911| 29086| 72850| Wyoming| 56| 571005| 1853|
 656093| 291473| 962293| Alabama| 01| 4813429| 14727|
 876194| 361041| 1254559| Indiana| 18| 6597701| 17324|
 204277| 89248| 298583| Idaho| 16| 1732106| 5058|
+--------------+--------------+----------------------+--------------------+--------+----------------+--------+
only showing top 20 rows

In [0]:
# change order of data frame
censusListdf = censusListdf.select('State', 'Total_Population', 'Public_Insurance_Alone', 'Medicare_Alone', 'Medicaid_Alone', 'VA_Alone', 'State_ID')

In [0]:
# view changes
censusListdf.show()

+--------------------+----------------+----------------------+--------------+--------------+--------+--------+
 State|Total_Population|Public_Insurance_Alone|Medicare_Alone|Medicaid_Alone|VA_Alone|State_ID|
+--------------------+----------------+----------------------+--------------+--------------+--------+--------+
District of Columbia| 692059| 154098| 17803| 135312| 983| 11|
 California| 38838726| 9774780| 1974723| 7724092| 75965| 06|
 Pennsylvania| 12590644| 2407050| 657439| 1727514| 22097| 42|
 New York| 19276809| 4542469| 932578| 3585778| 24113| 36|
 West Virginia| 1778080| 471938| 114398| 349027| 8513| 54|
 Utah| 3124563| 339098| 116851| 217665| 4582| 49|
 Alaska| 711104| 139179| 23270| 111808| 4101| 02|
 South Carolina| 4990992| 1009459| 306122| 684845| 18492| 45|
 Maine| 1325025| 227239| 78486| 143635| 5118| 23|
 Georgia| 10321846| 1851109| 542688| 1270484| 37937| 13|
 North Dakota| 743105| 84685| 26788| 56303| 1594| 38|
 Florida| 20897188| 4367261| 1739231| 2551095| 76935| 12|
 Arizona| 7065326| 1650936| 473869| 1153664| 23403| 04|
 New Hampshire| 1338372| 207861| 75884| 128469| 3508| 33|
 Louisiana| 4556953| 1263602| 280333| 968482| 14787| 22|
 Oregon| 4135531| 921871| 233948| 672968| 14955| 41|
 Wyoming| 571005| 72850| 29086| 41911| 1853| 56|
 Alabama| 4813429| 962293| 291473| 656093| 14727| 01|
 Indiana| 6597701| 1254559| 361041| 876194| 17324| 18|
 Idaho| 1732106| 298583| 89248| 204277| 5058| 16|
+--------------------+----------------+----------------------+--------------+--------------+--------+--------+
only showing top 20 rows

In [0]:
# check data types
censusListdf.printSchema()

root
-- State: string (nullable = true)
-- Total_Population: string (nullable = true)
-- Public_Insurance_Alone: string (nullable = true)
-- Medicare_Alone: string (nullable = true)
-- Medicaid_Alone: string (nullable = true)
-- VA_Alone: string (nullable = true)
-- State_ID: string (nullable = true)

In [0]:
# change data types for appropriate columns
# int: state id
# float: all population columns
# did not change data type (string) for state
# display dataframe and print schema to confirm changes accepted
censusListdf = censusListdf.withColumn('Total_Population', censusListdf.Total_Population.cast('float')).withColumn('Public_Insurance_Alone', censusListdf.Public_Insurance_Alone.cast('float')).withColumn('Medicare_Alone', censusListdf.Medicare_Alone.cast('float')).withColumn('Medicaid_Alone', censusListdf.Medicaid_Alone.cast('float')).withColumn('VA_Alone', censusListdf.VA_Alone.cast('float')).withColumn('State_ID', censusListdf.State_ID.cast('int'))
censusListdf.show()
# view finalized data frame
display(censusListdf)
censusListdf.printSchema()

+--------------------+----------------+----------------------+--------------+--------------+--------+--------+
 State|Total_Population|Public_Insurance_Alone|Medicare_Alone|Medicaid_Alone|VA_Alone|State_ID|
+--------------------+----------------+----------------------+--------------+--------------+--------+--------+
District of Columbia| 692059.0| 154098.0| 17803.0| 135312.0| 983.0| 11|
 California| 3.8838728E7| 9774780.0| 1974723.0| 7724092.0| 75965.0| 6|
 Pennsylvania| 1.2590644E7| 2407050.0| 657439.0| 1727514.0| 22097.0| 42|
 New York| 1.9276808E7| 4542469.0| 932578.0| 3585778.0| 24113.0| 36|
 West Virginia| 1778080.0| 471938.0| 114398.0| 349027.0| 8513.0| 54|
 Utah| 3124563.0| 339098.0| 116851.0| 217665.0| 4582.0| 49|
 Alaska| 711104.0| 139179.0| 23270.0| 111808.0| 4101.0| 2|
 South Carolina| 4990992.0| 1009459.0| 306122.0| 684845.0| 18492.0| 45|
 Maine| 1325025.0| 227239.0| 78486.0| 143635.0| 5118.0| 23|
 Georgia| 1.0321846E7| 1851109.0| 542688.0| 1270484.0| 37937.0| 13|
 North Dakota| 743105.0| 84685.0| 26788.0| 56303.0| 1594.0| 38|
 Florida| 2.0897188E7| 4367261.0| 1739231.0| 2551095.0| 76935.0| 12|
 Arizona| 7065326.0| 1650936.0| 473869.0| 1153664.0| 23403.0| 4|
 New Hampshire| 1338372.0| 207861.0| 75884.0| 128469.0| 3508.0| 33|
 Louisiana| 4556953.0| 1263602.0| 280333.0| 968482.0| 14787.0| 22|
 Oregon| 4135531.0| 921871.0| 233948.0| 672968.0| 14955.0| 41|
 Wyoming| 571005.0| 72850.0| 29086.0| 41911.0| 1853.0| 56|
 Alabama| 4813429.0| 962293.0| 291473.0| 656093.0| 14727.0| 1|
 Indiana| 6597701.0| 1254559.0| 361041.0| 876194.0| 17324.0| 18|
 Idaho| 1732106.0| 298583.0| 89248.0| 204277.0| 5058.0| 16|
+--------------------+----------------+----------------------+--------------+--------------+--------+--------+
only showing top 20 rows

State,Total_Population,Public_Insurance_Alone,Medicare_Alone,Medicaid_Alone,VA_Alone,State_ID
District of Columbia,692059.0,154098.0,17803.0,135312.0,983.0,11
California,3.8838728E7,9774780.0,1974723.0,7724092.0,75965.0,6
Pennsylvania,1.2590644E7,2407050.0,657439.0,1727514.0,22097.0,42
New York,1.9276808E7,4542469.0,932578.0,3585778.0,24113.0,36
West Virginia,1778080.0,471938.0,114398.0,349027.0,8513.0,54
Utah,3124563.0,339098.0,116851.0,217665.0,4582.0,49
Alaska,711104.0,139179.0,23270.0,111808.0,4101.0,2
South Carolina,4990992.0,1009459.0,306122.0,684845.0,18492.0,45
Maine,1325025.0,227239.0,78486.0,143635.0,5118.0,23
Georgia,1.0321846E7,1851109.0,542688.0,1270484.0,37937.0,13


root
-- State: string (nullable = true)
-- Total_Population: float (nullable = true)
-- Public_Insurance_Alone: float (nullable = true)
-- Medicare_Alone: float (nullable = true)
-- Medicaid_Alone: float (nullable = true)
-- VA_Alone: float (nullable = true)
-- State_ID: integer (nullable = true)

In [0]:
# rename for SQL
censusListdf = censusListdf.withColumnRenamed("State","Census_State").withColumnRenamed("Public_Insurance_Alone","Population_Publicly_Insured").withColumnRenamed("Medicare_Alone","Population_Medicare").withColumnRenamed("Medicaid_Alone","Population_Medicaid").withColumnRenamed("VA_Alone","Population_VA")

In [0]:
# add Census_ID for SQL DB
from pyspark.sql.window import Window as W
from pyspark.sql import functions as F
df = censusListdf.withColumn("Census_ID", F.monotonically_increasing_id())

display(df)

Census_State,Total_Population,Population_Publicly_Insured,Population_Medicare,Population_Medicaid,Population_VA,State_ID,Census_ID
District of Columbia,692059.0,154098.0,17803.0,135312.0,983.0,11,0
California,3.8838728E7,9774780.0,1974723.0,7724092.0,75965.0,6,1
Pennsylvania,1.2590644E7,2407050.0,657439.0,1727514.0,22097.0,42,2
New York,1.9276808E7,4542469.0,932578.0,3585778.0,24113.0,36,3
West Virginia,1778080.0,471938.0,114398.0,349027.0,8513.0,54,4
Utah,3124563.0,339098.0,116851.0,217665.0,4582.0,49,5
Alaska,711104.0,139179.0,23270.0,111808.0,4101.0,2,6
South Carolina,4990992.0,1009459.0,306122.0,684845.0,18492.0,45,7
Maine,1325025.0,227239.0,78486.0,143635.0,5118.0,23,8
Georgia,1.0321846E7,1851109.0,542688.0,1270484.0,37937.0,13,9


In [0]:
# correct order of Census_ID
from pyspark.sql.window import Window as W
from pyspark.sql import functions as F

windowSpec = W.orderBy("Census_State")
censusListdf = df.withColumn("Census_ID", F.row_number().over(windowSpec))
censusListdf.show()

+--------------------+----------------+---------------------------+-------------------+-------------------+-------------+--------+---------+
 Census_State|Total_Population|Population_Publicly_Insured|Population_Medicare|Population_Medicaid|Population_VA|State_ID|Census_ID|
+--------------------+----------------+---------------------------+-------------------+-------------------+-------------+--------+---------+
 Alabama| 4813429.0| 962293.0| 291473.0| 656093.0| 14727.0| 1| 1|
 Alaska| 711104.0| 139179.0| 23270.0| 111808.0| 4101.0| 2| 2|
 Arizona| 7065326.0| 1650936.0| 473869.0| 1153664.0| 23403.0| 4| 3|
 Arkansas| 2957271.0| 800142.0| 199507.0| 588981.0| 11654.0| 5| 4|
 California| 3.8838728E7| 9774780.0| 1974723.0| 7724092.0| 75965.0| 6| 5|
 Colorado| 5588760.0| 1097732.0| 296328.0| 782875.0| 18529.0| 8| 6|
 Connecticut| 3520172.0| 739685.0| 191001.0| 543381.0| 5303.0| 9| 7|
 Delaware| 951930.0| 193141.0| 50174.0| 140917.0| 2050.0| 10| 8|
District of Columbia| 692059.0| 154098.0| 17803.0| 135312.0| 983.0| 11| 9|
 Florida| 2.0897188E7| 4367261.0| 1739231.0| 2551095.0| 76935.0| 12| 10|
 Georgia| 1.0321846E7| 1851109.0| 542688.0| 1270484.0| 37937.0| 13| 11|
 Hawaii| 1365172.0| 235983.0| 58984.0| 174072.0| 2927.0| 15| 12|
 Idaho| 1732106.0| 298583.0| 89248.0| 204277.0| 5058.0| 16| 13|
 Illinois| 1.2536614E7| 2565191.0| 668351.0| 1873803.0| 23037.0| 17| 14|
 Indiana| 6597701.0| 1254559.0| 361041.0| 876194.0| 17324.0| 18| 15|
 Iowa| 3106492.0| 572162.0| 146704.0| 418151.0| 7307.0| 19| 16|
 Kansas| 2853649.0| 411026.0| 141075.0| 262042.0| 7909.0| 20| 17|
 Kentucky| 4379339.0| 1137796.0| 255753.0| 870125.0| 11918.0| 21| 18|
 Louisiana| 4556953.0| 1263602.0| 280333.0| 968482.0| 14787.0| 22| 19|
 Maine| 1325025.0| 227239.0| 78486.0| 143635.0| 5118.0| 23| 20|
+--------------------+----------------+---------------------------+-------------------+-------------------+-------------+--------+---------+
only showing top 20 rows

In [0]:
# change order of data frame
censusListdf = censusListdf.select('Census_ID', 'Census_State', 'State_ID', 'Total_Population', 'Population_Publicly_Insured', 'Population_Medicare', 'Population_Medicaid', 'Population_VA')

In [0]:
display(censusListdf)

Census_ID,Census_State,State_ID,Total_Population,Population_Publicly_Insured,Population_Medicare,Population_Medicaid,Population_VA
1,Alabama,1,4813429.0,962293.0,291473.0,656093.0,14727.0
2,Alaska,2,711104.0,139179.0,23270.0,111808.0,4101.0
3,Arizona,4,7065326.0,1650936.0,473869.0,1153664.0,23403.0
4,Arkansas,5,2957271.0,800142.0,199507.0,588981.0,11654.0
5,California,6,3.8838728E7,9774780.0,1974723.0,7724092.0,75965.0
6,Colorado,8,5588760.0,1097732.0,296328.0,782875.0,18529.0
7,Connecticut,9,3520172.0,739685.0,191001.0,543381.0,5303.0
8,Delaware,10,951930.0,193141.0,50174.0,140917.0,2050.0
9,District of Columbia,11,692059.0,154098.0,17803.0,135312.0,983.0
10,Florida,12,2.0897188E7,4367261.0,1739231.0,2551095.0,76935.0


In [0]:
# establish connection to COPY database
database = "Copy_teamRocket"
table = "dbo.CensusVals"
user = "gen10dbadmin"
password  = "vbtwrEmgSG12mabBr9ReZkcPrrDbTR-Y"
server = "gen10-data-fundamentals-22-02-sql-server.database.windows.net"


In [0]:
# write census table to COPY database
censusListdf.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("overwrite") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [0]:
# # establish connection to database
# database = "teamRocket"
# table = "dbo.CensusVals"
# user = "rocketuser"
# password  = "Bl@sting0ff"
# server = "gen10-data-fundamentals-22-02-sql-server.database.windows.net"


In [0]:
# # write census table to database
# censusListdf.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
#     .mode("overwrite") \
#     .option("dbtable", table) \
#     .option("user", user) \
#     .option("password", password) \
#     .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
#     .save()